In [1]:
import os
import sys
import time
import pandas as pd
import datetime
#import pandas.io.data as web
from pandas_datareader import data
import matplotlib.pyplot as plt
from matplotlib import style
import glob
import numpy as np

# Build train and test datasets

In [2]:
# Load csvs and create train, test datasets
def get_ticker_idx_maps(dfs):
    sorted_keys = sorted(dfs.keys())
    ticker_idx = {ticker: i for i,ticker in enumerate(sorted_keys)}
    idx_ticker = {i: ticker for i,ticker in enumerate(sorted_keys)}
    return ticker_idx, idx_ticker
    
def dict_2d_to_3d(dfs, ticker_idx, colname_idx, shift_value):
    sorted_keys = sorted(dfs.keys())
    # dfs with tickers in keys -> dfs with keys as index(tickers)
    dfs_int_keys = {ticker_idx[ticker]: dfs[ticker].values for ticker in sorted_keys}
    # then convert it to 3d array
    X = np.array(list(np.array(pd.DataFrame.from_dict(dfs_int_keys[key])) for key in sorted(dfs_int_keys.keys())))
    # y = np.roll(x,1,axis=1)
    # y[:,0] = 0
    y = np.array(list(dfs_int_keys[key][:,colname_idx['Adj Close']] for key in sorted(dfs_int_keys.keys())))
    #print("before shifting: y[0][:", shift_value, '] =', y[0][:shift_value])
    #print("before shifting: y[0][-", shift_value, ':] =', y[0][-shift_value:])
    for i in range(len(y)):
        y[i] = np.roll(y[i], -shift_value, axis=0)
        # The last shift_value rows have no data
        #y[i][-shift_value:] = None
        
        # Remove the last shift_value elements of the arrays
        y[i] = y[i][:len(y[i])-shift_value].copy()
        X[i] = X[i][:len(X[i])-shift_value].copy()
   
    #print(" X[0][:", shift_value, "]", X[0][:shift_value])
    #print('After shifting: y[0][0]', y[0][0])
    #print(' y[len(y)-shift_value-1:]', y[len(y)-shift_value-1:])

    return(X, y) #np.dstack(temp)


def load_csv_data(source_dir="../data", days_for_prediction=30):
    '''
    Input:
    - source_dir: directory where the stock price CSVs are located
    - days_for_prediction: number of days for the prediction prices. Must be at least 30 days
    Description:
    Read csv files in source_dir, load into dataframes and split into
    X_train, Y_train, X_test, Y_test
    '''
    assert (days_for_prediction >= 30), "days_for_prediction must be >= 30"

    csv_file_pattern = os.path.join(source_dir, "*.csv")
    csv_files = glob.glob(csv_file_pattern)
    dfs1 = {}
    dfs2 = {}
    for filename in csv_files:
        arr = filename.split('/')
        ticker = arr[-1].split('.')[0]
        #print(filename, ticker)
        #print(ticker, df.head())
        
        #  Date, Open, High , Low , Close, Adj Close, Volume
        df = pd.read_csv(filename, parse_dates=[0]) #index_col='Date')
        
        # Split 80/20
        split_len = int(len(df) * .8)
        #print(split_len)
        df1 = df.iloc[:split_len,:]
        df2 = df.iloc[split_len:,:]
        #print("df1.tail", df1.tail())
        #print("df2.head", df2.head())
        dfs1[ticker] = df1
        dfs2[ticker] = df2

        #print(len(df), len(df1)/len(df), len(df2)/len(df))
    
    colname_idx = {colname: i for i, colname in enumerate(list(dfs1[ticker]))}
    ticker_idx1, idx_ticker1 = get_ticker_idx_maps(dfs1)
    ticker_idx2, idx_ticker2 = get_ticker_idx_maps(dfs2)

    # Validate that the 2 mapping sets are the same for train and test data
    shared_items = set(ticker_idx1.items()) & set(ticker_idx2.items())
    assert(len(shared_items) == len(ticker_idx1) and len(shared_items) == len(ticker_idx2))
    
    # Convert dict of 2d arrays to 3d arrays
    train_X, train_y = dict_2d_to_3d(dfs1, ticker_idx1, colname_idx, days_for_prediction)
    test_X, test_y = dict_2d_to_3d(dfs2, ticker_idx2, colname_idx, days_for_prediction)
    return ticker_idx1, idx_ticker1, train_X, train_y, test_X, test_y

ticker_idx, idx_ticker, train_X, train_y, test_X, test_y = load_csv_data("../data/sp500") #, days_for_prediction=30)
#print(train_X, test_X)
print(train_X.shape, test_X.shape)
#print(train_X[0], test_X[0])

(501,) (501,)


In [3]:
# check shape for these 501 stocks
for i in range(len(train_X)):
    print(i, idx_ticker[i], train_X[i].shape)

0 A (3542, 7)
1 AAL (2389, 7)
2 AAP (3159, 7)
3 AAPL (3542, 7)
4 ABBV (926, 7)
5 ABC (3542, 7)
6 ABT (3542, 7)
7 ACN (3230, 7)
8 ADBE (3542, 7)
9 ADI (3542, 7)
10 ADM (3542, 7)
11 ADP (3542, 7)
12 ADS (3249, 7)
13 ADSK (3542, 7)
14 AEE (3542, 7)
15 AEP (3542, 7)
16 AES (3542, 7)
17 AET (3542, 7)
18 AFL (3542, 7)
19 AGN (3542, 7)
20 AIG (3542, 7)
21 AIV (3542, 7)
22 AIZ (2720, 7)
23 AJG (3542, 7)
24 AKAM (3542, 7)
25 ALB (3542, 7)
26 ALGN (3325, 7)
27 ALK (3542, 7)
28 ALL (3542, 7)
29 ALLE (749, 7)
30 ALXN (3542, 7)
31 AMAT (3542, 7)
32 AMD (3542, 7)
33 AME (3542, 7)
34 AMG (3542, 7)
35 AMGN (3542, 7)
36 AMP (2395, 7)
37 AMT (3542, 7)
38 AMZN (3542, 7)
39 ANDV (3542, 7)
40 ANSS (3542, 7)
41 ANTM (3176, 7)
42 AON (3542, 7)
43 AOS (3542, 7)
44 APA (3542, 7)
45 APC (3542, 7)
46 APD (3542, 7)
47 APH (3542, 7)
48 ARE (3542, 7)
49 ARNC (3542, 7)
50 ATVI (3542, 7)
51 AVB (3542, 7)
52 AVGO (1612, 7)
53 AVY (3542, 7)
54 AWK (1872, 7)
55 AXP (3542, 7)
56 AYI (3158, 7)
57 AZO (3542, 7)
58 BA (3542

### Note: Not all stocks' shapes are the same! 
Can we train this 3d array (501,,)?


In [4]:
#print('train_X[0][29:39]', train_X[0][29:39])
print(len(train_X[0]), len(train_y[0]))
print('train_X[0][30:40,5]', train_X[0][30:40,5])
# Should be same as above
print('train_y[0][:10]', train_y[0][:10])
assert(np.array_equal(train_X[0][30:40,5], train_y[0][:10]))


print('train_X[0][-40:,5]', train_X[0][-40:,5])
# Should be empty
print('train_y[0][-10:]', train_y[0][-10:])
print(train_y[0])

3542 3542
train_X[0][30:40,5] [52.183887 51.626209 61.823978 59.752552 58.318485 63.098698 68.038239
 68.914604 64.373398 66.205826]
train_y[0][:10] [52.183887 51.626209 61.823978 59.752552 58.318485 63.098698 68.038239
 68.914604 64.373398 66.205826]
train_X[0][-40:,5] [36.898647 36.843559 37.759476 37.917862 38.110703 37.766369 38.055607
 37.986744 38.041836 38.820019 39.866795 39.515579 39.226326 39.625763
 39.41227 39.570667 39.461605 39.565147 39.47541 38.798965 39.289051
 39.095768 39.654881 40.303711 40.317516 40.676445 40.676445 41.332191
 41.6497 41.760143 41.905094 42.001732 42.056946 41.049179 39.944782
 40.241585 40.10353900000001 39.792931 41.076797 40.138054]
train_y[0][-10:] [39.882652 39.896469 40.56601 40.324417 40.138054 39.247623 39.385674
 38.55048 38.267483 38.515968]
[52.183887 51.626209 61.823978 ..., 38.55048 38.267483 38.515968]


# Now try with baseline model